# Week 5 — Data Wrangling
### CSE 432/532 — Machine Learning

---

**Data wrangling** (sometimes called *data munging* or *data preparation*) is what you do to get raw data into a shape you can actually work with. Think of it as the grunt work before the fun stuff.

In practice, data is almost always messy — missing values, inconsistent formats, duplicates, outliers, data spread across different files. You need to deal with all of that before any model will give you reasonable results.

### The Data Wrangling Pipeline

| Step | Goal | Key Question |
|------|------|--------------|
| **1. Exploring** | Understand the data | *What do we have?* |
| **2. Structuring** | Uniform formats and scales | *Is everything comparable?* |
| **3. Cleaning** | Handle dirty data | *What's broken or missing?* |
| **4. Enriching** | Add value | *Can we add or derive features?* |
| 5. Validating | Verify consistency | *(Not covered today)* |
| 6. Publishing | Make available | *(Not covered today)* |

> **Today we'll work through steps 1-4, plus a quick intro to regex for text data.**

---

### Quick Note: None vs NaN vs NaT

| Symbol | Type | Use Case | Safe Check |
|--------|------|----------|------------|
| `None` | Python null | Any type | `is None` |
| `NaN` | Float | Numeric missing | `pd.isna()` |
| `NaT` | Datetime | Datetime missing | `pd.isna()` |

**Tip:** `pd.isna()` works for all three, so just use that as your go-to.

In [1]:
# ── Setup ──────────────────────────────────────────────
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

# Quick demo: pd.isna() works for None, NaN, and NaT
print("pd.isna(None):", pd.isna(None))
print("pd.isna(np.nan):", pd.isna(np.nan))
print("pd.isna(pd.NaT):", pd.isna(pd.NaT))
print("\n>> pd.isna() catches all three types of missing values!")

pd.isna(None): True
pd.isna(np.nan): True
pd.isna(pd.NaT): True

>> pd.isna() catches all three types of missing values!


In [2]:
# ── Build a small, realistic sample dataset ────────────
np.random.seed(42)
n = 30

departments = np.random.choice(['Engineering', 'Marketing', 'Sales', 'HR'], n)
levels = np.random.choice(['Junior', 'Mid', 'Senior'], n)

base_salary = {'Engineering': 90000, 'Marketing': 70000, 'Sales': 65000, 'HR': 68000}
level_mult = {'Junior': 0.8, 'Mid': 1.0, 'Senior': 1.3}

salaries = [
    int(base_salary[d] * level_mult[l] + np.random.normal(0, 5000))
    for d, l in zip(departments, levels)
]

df = pd.DataFrame({
    'Name':       [f'Emp_{i:03d}' for i in range(n)],
    'Department': departments,
    'Level':      levels,
    'Salary':     salaries,
    'YearsExp':   np.random.randint(1, 20, n).astype(float),
    'Rating':     np.round(np.random.uniform(2.0, 5.0, n), 1),
    'JoinDate':   pd.date_range('2015-01-01', periods=n, freq='45D').strftime('%m/%d/%Y'),
    'PartCode':   [f"{np.random.choice(['HW','SW','QA'])}-{np.random.randint(100,999)}" for _ in range(n)]
})

# ── Inject messiness (we'll clean it later!) ───────────
df.loc[3, 'Salary'] = np.nan          # missing
df.loc[7, 'Salary'] = np.nan          # missing
df.loc[12, 'YearsExp'] = np.nan       # missing
df.loc[20, 'Rating'] = np.nan         # missing
df.loc[5, 'Salary'] = 350000          # outlier!
df = pd.concat([df, df.iloc[[0]]], ignore_index=True)  # duplicate row

print(f"Dataset shape: {df.shape}")
df.head(10)

Dataset shape: (31, 8)


,Name,Department,Level,Salary,YearsExp,Rating,JoinDate,PartCode
0,Emp_000,Sales,Junior,51298.0,13.0,3.4,01/01/2015,QA-739
1,Emp_001,HR,Junior,54234.0,1.0,2.4,02/15/2015,SW-447
2,Emp_002,Engineering,Mid,86254.0,7.0,4.1,04/01/2015,HW-330
3,Emp_003,Sales,Mid,NaN,9.0,4.3,05/16/2015,SW-324
4,Emp_004,Sales,Junior,56744.0,1.0,3.7,06/30/2015,HW-476
5,Emp_005,HR,Junior,350000.0,12.0,4.3,08/14/2015,QA-732
6,Emp_006,Engineering,Junior,70159.0,8.0,3.5,09/28/2015,HW-844
7,Emp_007,Engineering,Senior,NaN,11.0,3.6,11/12/2015,QA-458
8,Emp_008,Sales,Senior,78534.0,19.0,3.3,12/27/2015,SW-555
9,Emp_009,Marketing,Senior,88954.0,17.0,2.1,02/10/2016,QA-748


In [3]:
# ── Quick overview ─────────────────────────────────────
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Name        31 non-null     str    
 1   Department  31 non-null     str    
 2   Level       31 non-null     str    
 3   Salary      29 non-null     float64
 4   YearsExp    30 non-null     float64
 5   Rating      30 non-null     float64
 6   JoinDate    31 non-null     str    
 7   PartCode    31 non-null     str    
dtypes: float64(3), str(5)
memory usage: 2.1 KB


In [4]:
df.describe()

,Salary,YearsExp,Rating
count,29.000000,30.000000,30.000000
mean,82278.000000,8.000000,3.506667
std,54034.957602,4.941032,0.842383
min,51298.000000,1.000000,2.100000
25%,58129.000000,3.500000,2.750000
50%,70159.000000,8.000000,3.500000
75%,88954.000000,11.750000,4.300000
max,350000.000000,19.000000,4.800000


---
# 1. Exploring Data

Before you start transforming anything, you should understand what you're working with. That means looking at patterns, distributions, and group-level summaries.

### Grouping with `groupby()`

`groupby()` splits your DataFrame into subsets based on some categorical column. You can then compute statistics on each group.

```python
df.groupby('CategoryColumn')['ValueColumn'].statistic()
```

Some things you might want to do:
- Compare mean salary across departments
- Count how many employees are at each level (a **frequency table**)
- Find the highest rating in each group

> **Before we run the code** — which department would you guess has the highest average salary? Why?

In [5]:
# ── Frequency table: count per department ──────────────
print("Frequency table (group sizes):")
print(df.groupby('Department').size())
print()

# Same thing with value_counts
print("Using value_counts():")
print(df['Department'].value_counts())

Frequency table (group sizes):
Department
Engineering     5
HR             10
Marketing       6
Sales          10
dtype: int64

Using value_counts():
Department
Sales          10
HR             10
Marketing       6
Engineering     5
Name: count, dtype: int64


In [6]:
# ── Mean salary by department ──────────────────────────
df.groupby('Department')['Salary'].mean()

Department
Engineering    89580.000000
HR             96894.600000
Marketing      77678.666667
Sales          65858.222222
Name: Salary, dtype: float64

In [7]:
# ── Multiple aggregations at once ─────────────────────
df.groupby('Department')['Salary'].agg(['mean', 'median', 'std', 'count'])

,mean,median,std,count
Department,,,,
Engineering,89580.000000,85517.0,19749.385864,4
HR,96894.600000,64668.5,89845.127196,10
Marketing,77678.666667,79718.0,17394.528791,6
Sales,65858.222222,62766.0,13776.471326,9


### IN-CLASS ACTIVITY -- TODO #1

Using `groupby()`, compute the **median Rating** grouped by **Level** (Junior, Mid, Senior).

*Hint: You just saw `df.groupby('Department')['Salary'].mean()` — just swap the column names and the statistic.*

In [35]:
# TODO #1: Compute the median Rating grouped by Level
# Your code here (1 line):
df.groupby('Level')['Rating'].median()

Level
Junior    4.0
Mid       4.1
Senior    2.9
Name: Rating, dtype: float64

In [36]:
# ── dropna parameter in groupby ────────────────────────
# If grouping feature has NaN, dropna controls behavior
df_temp = df.copy()
df_temp.loc[0, 'Department'] = np.nan  # inject a NaN in grouping column

print("dropna=True (default) — NaN group excluded:")
print(df_temp.groupby('Department', dropna=True).size())
print("\ndropna=False — NaN gets its own group:")
print(df_temp.groupby('Department', dropna=False).size())

dropna=True (default) — NaN group excluded:
Department
Engineering     5
HR             10
Marketing       6
Sales           9
dtype: int64

dropna=False — NaN gets its own group:
Department
Engineering     5
HR             10
Marketing       6
Sales           9
NaN             1
dtype: int64


---
### Pivot Tables with `pivot_table()`

A pivot table is like `groupby()` but on **two** categorical features at once, laid out as a 2D matrix. Makes it much easier to read.

- One feature goes to **rows** (`index`)
- Another goes to **columns** (`columns`)
- A numeric feature provides the **values** (`values`)
- You pick an aggregation function with `aggfunc` (default is mean)

A **contingency table** is just a pivot table where `aggfunc='count'` — it tells you how many observations fall into each combination.

```python
df.pivot_table(values='Salary', index='Department', columns='Level', aggfunc='mean')
```

| | `groupby()` | `pivot_table()` |
|---|---|---|
| **Groups by** | 1+ features | Exactly 2 (rows & columns) |
| **Output** | Series / DataFrame | 2D cross-tabulation |
| **Best for** | Flexible aggregation | Readable matrix display |

In [37]:
# ── Pivot Table: Mean Salary by Department × Level ────
pivot_mean = df.pivot_table(
    values='Salary',
    index='Department',
    columns='Level',
    aggfunc='mean'
)
pivot_mean

Level,Junior,Mid,Senior
Department,,,
Engineering,70159.000000,85517.000000,117127.000000
HR,117232.200000,62997.000000,85597.000000
Marketing,58458.500000,70482.000000,92891.000000
Sales,53113.333333,62470.666667,81990.666667


### IN-CLASS ACTIVITY -- TODO #2

Create a **contingency table** (counts) using `pivot_table()`. Use the same `values`, `index`, and `columns` as above, but change `aggfunc` to count instead of mean.

*Hint: Just change `aggfunc='mean'` to `aggfunc='count'`.*

In [40]:
# TODO #2: Create a contingency table (counts) from the pivot table above.
# Change the aggfunc parameter.
# Your code here (1 line):
counts = df.pivot_table(values='Salary', index='Department', columns='Level', aggfunc='count')
counts

Level,Junior,Mid,Senior
Department,,,
Engineering,1,2,1
HR,5,2,3
Marketing,2,1,3
Sales,3,3,3


In [41]:
# ── Pivot with multiple aggfuncs ──────────────────────
df.pivot_table(
    values='Salary',
    index='Department',
    columns='Level',
    aggfunc=['mean', 'count']
)

mean                               count           
Level               Junior           Mid         Senior Junior Mid Senior
Department                                                               
Engineering   70159.000000  85517.000000  117127.000000      1   2      1
HR           117232.200000  62997.000000   85597.000000      5   2      3
Marketing     58458.500000  70482.000000   92891.000000      2   1      3
Sales         53113.333333  62470.666667   81990.666667      3   3      3

---
# 2. Structuring Data

The goal here is to make all your data comparable — uniform formats, consistent units, proper types. If your data is in different scales or formats, downstream analysis will be unreliable.

Three things we'll cover:
1. **Formatting** — Getting uniform types, date formats, string casing
2. **Feature Scaling** — Standardization (z-score) and Normalization (min-max)
3. **Unpacking** — Splitting overloaded features into simpler ones

---

### 2a. Formatting

Every column should use a consistent format:
- Dates should be `datetime` type, not strings
- Strings should use consistent casing (`Title Case` or `lowercase`)
- Numbers should be in consistent units (all meters, all dollars, etc.)

In [13]:
# ── Convert JoinDate from string to datetime ───────────
print(f"JoinDate type BEFORE: {df['JoinDate'].dtype}")
df['JoinDate'] = pd.to_datetime(df['JoinDate'])
print(f"JoinDate type AFTER:  {df['JoinDate'].dtype}")
print()
df[['Name', 'JoinDate']].head()

JoinDate type BEFORE: str
JoinDate type AFTER:  datetime64[us]



,Name,JoinDate
0,Emp_000,2015-01-01
1,Emp_001,2015-02-15
2,Emp_002,2015-04-01
3,Emp_003,2015-05-16
4,Emp_004,2015-06-30


### IN-CLASS ACTIVITY -- TODO #3

Given the messy Series below, convert all values to **lowercase** using a pandas string method.

*Hint: pandas Series have a `.str` accessor with methods like `.lower()`, `.upper()`, `.title()`.*

In [44]:
# TODO #3: Convert all values to lowercase
messy_names = pd.Series(['ENGINEERING', 'marketing', 'Sales', 'h.r.'])
print("Before:", messy_names.tolist())

result = messy_names.str.lower()
print("After:", result.tolist())
# Your code here (1 line):
# result = messy_names.___________
# print("After:", result.tolist())

Before: ['ENGINEERING', 'marketing', 'Sales', 'h.r.']
After: ['engineering', 'marketing', 'sales', 'h.r.']


### 2b. Feature Scaling

Numeric features often live on completely different scales. Salary might be in the tens of thousands while Rating is between 2 and 5. Many ML algorithms (especially distance-based ones like KNN) get thrown off by this.

| Method | Formula | Range | When to use |
|--------|---------|-------|-------------|
| **Standardization** (z-score) | $z = \frac{x - \mu}{\sigma}$ | Centered at 0 | Default choice; handles outliers better |
| **Normalization** (min-max) | $x' = \frac{x - x_{min}}{x_{max} - x_{min}}$ | [0, 1] | When you need bounded inputs |

> **Key point:** Outliers crush min-max normalization because they stretch the range. Standardization is more robust since it uses mean and std instead of min/max.

In [15]:
# ── The scale problem ──────────────────────────────────
print("BEFORE scaling — look at the different ranges:")
print(f"  Salary range:   {df['Salary'].min():>10,.0f} — {df['Salary'].max():>10,.0f}")
print(f"  YearsExp range: {df['YearsExp'].min():>10.1f} — {df['YearsExp'].max():>10.1f}")
print(f"  Rating range:   {df['Rating'].min():>10.1f} — {df['Rating'].max():>10.1f}")
print("\nNote: Salary is ~10,000x larger than Rating — this matters for distance-based algorithms!")

BEFORE scaling — look at the different ranges:
  Salary range:       51,298 —    350,000
  YearsExp range:        1.0 —       19.0
  Rating range:          2.1 —        4.8

Note: Salary is ~10,000x larger than Rating — this matters for distance-based algorithms!


In [16]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Work with non-null rows for scaling demo
cols = ['Salary', 'YearsExp', 'Rating']
df_numeric = df[cols].dropna()

# ── Standardization (z-score) ─────────────────────────
z_scaler = StandardScaler()
z_scores = pd.DataFrame(
    z_scaler.fit_transform(df_numeric),
    columns=[c + '_z' for c in cols],
    index=df_numeric.index
)

# ── Normalization (min-max) ───────────────────────────
mm_scaler = MinMaxScaler()
normalized = pd.DataFrame(
    mm_scaler.fit_transform(df_numeric),
    columns=[c + '_norm' for c in cols],
    index=df_numeric.index
)

# ── Side-by-side comparison ───────────────────────────
comparison = pd.concat([df_numeric.head(6), z_scores.head(6), normalized.head(6)], axis=1)
print("Original  →  Standardized (z-score)  →  Normalized (min-max)")
comparison

Original  →  Standardized (z-score)  →  Normalized (min-max)


,Salary,YearsExp,Rating,Salary_z,YearsExp_z,Rating_z,Salary_norm,YearsExp_norm,Rating_norm
0,51298.0,13.0,3.4,-0.592428,1.012954,-0.069331,0.000000,0.666667,0.481481
1,54234.0,1.0,2.4,-0.538792,-1.348176,-1.239292,0.009829,0.000000,0.111111
2,86254.0,7.0,4.1,0.046167,-0.167611,0.749642,0.117026,0.333333,0.740741
4,56744.0,1.0,3.7,-0.492938,-1.348176,0.281657,0.018232,0.000000,0.592593
5,350000.0,12.0,4.3,4.864418,0.816193,0.983634,1.000000,0.611111,0.814815
6,70159.0,8.0,3.5,-0.247865,0.029150,0.047665,0.063143,0.388889,0.518519


In [17]:
# ── Notice: the outlier (Salary=350000) affects normalization! ──
print("Normalized Salary stats:")
print(normalized['Salary_norm'].describe().round(3))
print("\nNote: The outlier pushes most normalized salaries close to 0!")
print("  Meanwhile, z-scores spread values around the mean regardless of outliers.")

Normalized Salary stats:
count    27.000
mean      0.109
std       0.187
min       0.000
25%       0.023
50%       0.064
75%       0.127
max       1.000
Name: Salary_norm, dtype: float64

Note: The outlier pushes most normalized salaries close to 0!
  Meanwhile, z-scores spread values around the mean regardless of outliers.


### 2c. Unpacking Overloaded Features

Sometimes a single column packs multiple pieces of information into one value. For example, `PartCode = 'SW-482'` has both a **role type** (`SW`) and an **ID** (`482`) crammed together.

It's better to **unpack** these into separate columns. You can keep the original around as a reference.

In [18]:
# ── Unpack PartCode into RoleType and PartID ──────────
print("Original PartCode samples:")
print(df['PartCode'].head(6).tolist())

# Method 1: str.split() + str accessor
df['RoleType'] = df['PartCode'].str.split('-').str[0]
df['PartID']   = df['PartCode'].str.split('-').str[1]

# Method 2 (alternative): string slicing
# df['RoleType'] = df['PartCode'].str[:2]

# Insert could also be used:
# df.insert(loc=8, column='RoleType', value=df['PartCode'].str.split('-').str[0])

print("\nAfter unpacking:")
df[['PartCode', 'RoleType', 'PartID']].head(6)

Original PartCode samples:
['QA-739', 'SW-447', 'HW-330', 'SW-324', 'HW-476', 'QA-732']

After unpacking:


,PartCode,RoleType,PartID
0,QA-739,QA,739
1,SW-447,SW,447
2,HW-330,HW,330
3,SW-324,SW,324
4,HW-476,HW,476
5,QA-732,QA,732


---
# 3. Cleaning Data

This is where we deal with **dirty data** — missing values, outliers, and duplicates. If you skip this step, your model will either crash or learn from garbage.

### Types of Dirty Data

| Type | What it looks like | In Python | Why it matters |
|------|-----------|----------------------|--------|
| **Missing** | Unknown or blank values | `NaN`, `None`, `NaT` | Algorithms crash or produce biased results |
| **Outliers** | Extreme values (2-3+ standard deviations out) | Normal values, just really far from the rest | Skew your statistics and mess up training |
| **Duplicates** | Identical rows | Exact copies | Over-represent certain data points |

### Two Approaches

| Strategy | When to use | Key methods |
|----------|-------------|-------------|
| **Discard** | Small % missing, data is missing at random, or duplicates | `dropna()`, `drop_duplicates()`, `drop()` |
| **Impute** | You want to keep all your rows/columns | `fillna()`, `replace()` |

In [19]:
# ── Step 1: Find missing data ─────────────────────────
print("Missing values per column:")
print(df.isnull().sum())
print(f"\nTotal missing cells: {df.isnull().sum().sum()} out of {df.size}")

Missing values per column:
Name          0
Department    0
Level         0
Salary        2
YearsExp      1
Rating        1
JoinDate      0
PartCode      0
RoleType      0
PartID        0
dtype: int64

Total missing cells: 4 out of 310


### IN-CLASS ACTIVITY -- TODO #4

Count the **number of duplicate rows** in the DataFrame.

*Hint: `df.duplicated()` gives you a boolean mask. How do you count `True` values?*

In [48]:
# TODO #4: Count the number of duplicate rows in df.
# Your code here (1 line):
print(f"Repeated values: {df.duplicated().sum()}")

Repeated values: 1


In [21]:
# ── Step 3: Find outliers using z-scores ──────────────
salary_mean = df['Salary'].mean()
salary_std  = df['Salary'].std()

# Outlier = value more than 2 standard deviations from mean
outlier_mask = (df['Salary'] - salary_mean).abs() > 2 * salary_std
print(f"Salary outliers (> 2σ from mean): {outlier_mask.sum()}")
df[outlier_mask][['Name', 'Department', 'Salary']]

Salary outliers (> 2σ from mean): 1


,Name,Department,Salary
5,Emp_005,HR,350000.0


### Discarding Strategies

| Method | When to use |
|--------|-------------|
| **Discard instances** (listwise deletion) | Small percentage missing, missingness is random |
| **Discard features** | More than ~60% of a column's values are missing |
| **Pairwise discarding** | Only drop for analyses that touch the dirty features (less common) |

For **duplicates**, you almost always want to remove them and keep just one copy.

In [22]:
# ── Discarding: remove duplicates ─────────────────────
df_clean = df.copy()
print(f"Shape BEFORE drop_duplicates: {df_clean.shape}")
df_clean = df_clean.drop_duplicates()
print(f"Shape AFTER  drop_duplicates: {df_clean.shape}")

Shape BEFORE drop_duplicates: (31, 10)
Shape AFTER  drop_duplicates: (30, 10)


In [23]:
# ── Discarding: dropna() removes rows with any NaN ────
print(f"Shape BEFORE dropna: {df_clean.shape}")
df_dropped = df_clean.dropna()
print(f"Shape AFTER  dropna: {df_dropped.shape}")
print(f"Lost {len(df_clean) - len(df_dropped)} rows")

# You can also drop a column with too many missing values:
# df_clean = df_clean.drop(columns=['SomeColumn'])

Shape BEFORE dropna: (30, 10)
Shape AFTER  dropna: (26, 10)
Lost 4 rows


### Imputing Missing and Outlier Values

Instead of throwing away rows, you can **replace** the bad values with something reasonable:

| Method | How it works | Best when |
|--------|-------------|-----------|
| **Mean imputation** | Fill with the column mean | Numeric data, roughly normal distribution |
| **Hot-deck** | Pick a random value from the same dataset | Less common these days |
| **Regression imputation** | Predict the missing value from other features | The feature correlates with others |

For **outliers**, the typical approach is to first flag them as NaN, then impute just like any other missing value.

In [24]:
# ── Imputation demo (instead of dropping rows) ────────
df_imputed = df_clean.copy()

# Step 1: Replace outliers with NaN
salary_mean = df_imputed['Salary'].mean()
salary_std  = df_imputed['Salary'].std()
outlier_mask = (df_imputed['Salary'] - salary_mean).abs() > 2 * salary_std
print(f"Replacing {outlier_mask.sum()} outlier(s) with NaN")
df_imputed.loc[outlier_mask, 'Salary'] = np.nan

# Step 2: Mean imputation for all numeric columns with missing values
for col in ['Salary', 'YearsExp', 'Rating']:
    n_missing = df_imputed[col].isnull().sum()
    if n_missing > 0:
        col_mean = df_imputed[col].mean()
        df_imputed[col] = df_imputed[col].fillna(col_mean)
        print(f"  Filled {n_missing} NaN(s) in '{col}' with mean = {col_mean:,.1f}")

print(f"\nRemaining missing values: {df_imputed.isnull().sum().sum()}")
print(f"Shape preserved: {df_imputed.shape} (no rows lost!)")

Replacing 1 outlier(s) with NaN
  Filled 3 NaN(s) in 'Salary' with mean = 73,509.8
  Filled 1 NaN(s) in 'YearsExp' with mean = 7.8
  Filled 1 NaN(s) in 'Rating' with mean = 3.5

Remaining missing values: 0
Shape preserved: (30, 10) (no rows lost!)


---
# 4. Enriching Data

Now we add information — either by **appending** external data or by **deriving** new features from what we already have.

### Appending Data

| Method | What it does | Typical use |
|--------|-------------|----------|
| `pd.concat([df1, df2], axis=0)` | Stack rows vertically | Adding new observations |
| `pd.concat([df1, df2], axis=1)` | Stack columns side by side | Adding new features (must share same index) |
| `df.merge(right, on='key')` | SQL-style join on a key | Linking related tables |

### Merge / Join Types

```
         Left Table        Right Table
INNER    |  A ^ B |        Only matching rows
LEFT     |  A     |<- B    All left rows, matching from right
RIGHT    |  A ->  |  B     All right rows, matching from left
OUTER    |  A U B |        Everything, NaN where no match
```

> **Good sources for external data:** Kaggle, FiveThirtyEight, data.gov, data.worldbank.org

In [25]:
# ── Concat: append new rows ───────────────────────────
new_employees = pd.DataFrame({
    'Name': ['Emp_100', 'Emp_101'],
    'Department': ['Engineering', 'Legal'],  # Legal is new!
    'Level': ['Junior', 'Senior'],
    'Salary': [75000, 95000],
    'YearsExp': [1.0, 12.0],
    'Rating': [4.2, 3.8]
})

df_appended = pd.concat([df_imputed, new_employees], ignore_index=True)
print(f"Before concat: {len(df_imputed)} rows")
print(f"After  concat: {len(df_appended)} rows")
print("\nNote: New rows have NaN for columns not in the new data:")
df_appended.tail(3)

Before concat: 30 rows
After  concat: 32 rows

Note: New rows have NaN for columns not in the new data:


,Name,Department,Level,Salary,YearsExp,Rating,JoinDate,PartCode,RoleType,PartID
29,Emp_029,HR,Mid,62760.0,3.0,4.7,2018-07-29,SW-771,SW,771
30,Emp_100,Engineering,Junior,75000.0,1.0,4.2,NaT,NaN,NaN,NaN
31,Emp_101,Legal,Senior,95000.0,12.0,3.8,NaT,NaN,NaN,NaN


In [26]:
# ── Merge: join with external data on a matching key ──
budget_df = pd.DataFrame({
    'Department': ['Engineering', 'Marketing', 'Sales', 'HR', 'Legal'],
    'AnnualBudget': [5_000_000, 2_000_000, 3_000_000, 1_500_000, 800_000],
    'HeadcountCap': [50, 25, 30, 15, 10]
})
print("External dataset:")
print(budget_df)

# Left join: keep all employees, add budget info where available
df_merged = df_imputed.merge(budget_df, on='Department', how='left')
print(f"\nAfter merge: {df_imputed.shape[1]} cols → {df_merged.shape[1]} cols")
df_merged[['Name', 'Department', 'Salary', 'AnnualBudget', 'HeadcountCap']].head(6)

External dataset:
    Department  AnnualBudget  HeadcountCap
0  Engineering       5000000            50
1    Marketing       2000000            25
2        Sales       3000000            30
3           HR       1500000            15
4        Legal        800000            10

After merge: 10 cols → 12 cols


,Name,Department,Salary,AnnualBudget,HeadcountCap
0,Emp_000,Sales,51298.000000,3000000,30
1,Emp_001,HR,54234.000000,1500000,15
2,Emp_002,Engineering,86254.000000,5000000,50
3,Emp_003,Sales,73509.777778,3000000,30
4,Emp_004,Sales,56744.000000,3000000,30
5,Emp_005,HR,73509.777778,1500000,15


In [27]:
# ── Compare join types ────────────────────────────────
# Our employee df has 4 departments; budget_df has 5 (includes 'Legal')
for how in ['inner', 'left', 'right', 'outer']:
    result = df_imputed.merge(budget_df, on='Department', how=how)
    depts = result['Department'].nunique()
    print(f"  {how:>5} join → {len(result):>3} rows, {depts} unique departments")

  inner join →  30 rows, 4 unique departments
   left join →  30 rows, 4 unique departments
  right join →  31 rows, 5 unique departments
  outer join →  31 rows, 5 unique departments


### Deriving New Features

You can also create new columns from the existing ones:
- **Calculate** something new (ratios, logs, etc.)
- **Bin** numeric values into categories with `pd.cut()`

This is where your **domain knowledge** really pays off — knowing what transformations make sense for your problem.

In [28]:
# ── Derive: compute new features ──────────────────────

# Ratio: Salary per year of experience
df_imputed['SalaryPerYear'] = (df_imputed['Salary'] / df_imputed['YearsExp']).round(0)

# apply() with a custom function
df_imputed['TenureYears'] = df_imputed['JoinDate'].apply(
    lambda d: round((pd.Timestamp.now() - d).days / 365.25, 1)
)

# Numeric → Categorical with pd.cut() (binning)
df_imputed['RatingCategory'] = pd.cut(
    df_imputed['Rating'],
    bins=[0, 3.0, 4.0, 5.0],
    labels=['Low', 'Medium', 'High']
)

df_imputed[['Name', 'Salary', 'SalaryPerYear', 'Rating', 'RatingCategory', 'TenureYears']].head(8)

,Name,Salary,SalaryPerYear,Rating,RatingCategory,TenureYears
0,Emp_000,51298.000000,3946.0,3.4,Medium,11.1
1,Emp_001,54234.000000,54234.0,2.4,Low,11.0
2,Emp_002,86254.000000,12322.0,4.1,High,10.9
3,Emp_003,73509.777778,8168.0,4.3,High,10.8
4,Emp_004,56744.000000,56744.0,3.7,Medium,10.7
5,Emp_005,73509.777778,6126.0,4.3,High,10.5
6,Emp_006,70159.000000,8770.0,3.5,Medium,10.4
7,Emp_007,73509.777778,6683.0,3.6,Medium,10.3


### IN-CLASS ACTIVITY -- TODO #5

Create a new column `LogSalary` that contains the **natural log** of `Salary`.

*Hint: Use `np.log` with `.apply()`, or just pass the column directly to `np.log()`.*

In [29]:
# TODO #5: Create a 'LogSalary' column using np.log on the Salary column.
# Your code here (1 line):


---
# 5. Regex and Text Matching in Python

**Regular expressions (regex)** let you write patterns to match, extract, and replace text. They're incredibly useful for:
- Cleaning up messy string columns
- Pulling structured info out of free text
- Validating formats (emails, phone numbers, etc.)

Python's built-in `re` module handles regex. Here are the patterns you'll use most:

### Core Regex Syntax

| Pattern | Meaning | Example |
|---------|---------|---------|
| `.` | Any character | `a.c` matches `abc`, `a1c` |
| `\d` | Any digit (0-9) | `\d{3}` matches `123` |
| `\w` | Word char (letter, digit, _) | `\w+` matches `hello_42` |
| `\s` | Whitespace | `\s+` matches spaces, tabs |
| `*` | 0 or more of previous | `ab*c` matches `ac`, `abbc` |
| `+` | 1 or more of previous | `ab+c` matches `abc`, `abbc` |
| `?` | 0 or 1 of previous | `colou?r` matches `color`, `colour` |
| `[]` | Character class | `[aeiou]` matches any vowel |
| `()` | Capture group | `(\d+)-(\w+)` lets you extract parts |
| `^` | Start of string | `^Hello` |
| `$` | End of string | `world$` |
| `\|` | OR | `cat\|dog` |

In [30]:
# ── Python re module basics ───────────────────────────
import re

text = "Contact us at support@company.com or sales@company.org. Call 555-123-4567."

# re.search() — find FIRST match
match = re.search(r'\d{3}-\d{3}-\d{4}', text)
if match:
    print(f"Phone found: {match.group()}")

# re.findall() — find ALL matches
emails = re.findall(r'[\w.]+@[\w.]+', text)
print(f"Emails found: {emails}")

# re.sub() — search and replace
redacted = re.sub(r'\d{3}-\d{3}-\d{4}', '[REDACTED]', text)
print(f"Redacted: {redacted}")

Phone found: 555-123-4567
Emails found: ['support@company.com', 'sales@company.org.']
Redacted: Contact us at support@company.com or sales@company.org. Call [REDACTED].


In [31]:
# ── Capture groups: extract structured parts ──────────
log_line = "2025-01-15 ERROR: File not found: /data/input.csv"

pattern = r'(\d{4}-\d{2}-\d{2})\s+(\w+):\s+(.+)'
match = re.search(pattern, log_line)

if match:
    print(f"Date:    {match.group(1)}")
    print(f"Level:   {match.group(2)}")
    print(f"Message: {match.group(3)}")

Date:    2025-01-15
Level:   ERROR
Message: File not found: /data/input.csv


### Regex with Pandas — Cleaning Text in DataFrames

Pandas string methods that support regex patterns:

| Task | Method | Example |
|------|--------|---------|
| Filter rows | `str.contains(pattern)` | Find all SW employees |
| Extract fields | `str.extract(pattern)` | Pull an email out of text |
| Replace text | `str.replace(pattern, repl)` | Strip out special characters |
| Split on pattern | `str.split(pattern)` | Split on commas or pipes |

In [32]:
# ── Regex with Pandas: extract from messy text ────────
messy_df = pd.DataFrame({
    'raw_input': [
        'John Smith, Age: 32, Email: john@gmail.com',
        'Jane Doe, Age: 28, Email: jane.doe@yahoo.com',
        'Bob Johnson, Age: 45, Email: bob_j@company.org',
        'Alice Brown, Age: 37, Email: alice@university.edu',
        'Charlie Wilson, Age: unknown, Email: N/A'
    ]
})
print("Raw messy data:")
print(messy_df)

# Extract structured fields using capture groups
messy_df['Name']  = messy_df['raw_input'].str.extract(r'^([A-Za-z ]+),')
messy_df['Age']   = messy_df['raw_input'].str.extract(r'Age:\s*(\d+)')
messy_df['Email'] = messy_df['raw_input'].str.extract(r'Email:\s*([\w.]+@[\w.]+)')

print("\nAfter regex extraction:")
messy_df[['Name', 'Age', 'Email']]

Raw messy data:
                                           raw_input
0         John Smith, Age: 32, Email: john@gmail.com
1       Jane Doe, Age: 28, Email: jane.doe@yahoo.com
2     Bob Johnson, Age: 45, Email: bob_j@company.org
3  Alice Brown, Age: 37, Email: alice@university.edu
4           Charlie Wilson, Age: unknown, Email: N/A

After regex extraction:


,Name,Age,Email
0,John Smith,32,john@gmail.com
1,Jane Doe,28,jane.doe@yahoo.com
2,Bob Johnson,45,bob_j@company.org
3,Alice Brown,37,alice@university.edu
4,Charlie Wilson,NaN,NaN


In [33]:
# ── str.replace(): clean money strings ────────────────
dirty_prices = pd.Series(['$1,200.00', '$850.50', '$3,400.75', 'N/A', '$12.00'])
print("Before:", dirty_prices.tolist())

clean_prices = dirty_prices.str.replace(r'[\$,]', '', regex=True)
clean_prices = pd.to_numeric(clean_prices, errors='coerce')
print("After: ", clean_prices.tolist())

Before: ['$1,200.00', '$850.50', '$3,400.75', 'N/A', '$12.00']
After:  [1200.0, 850.5, 3400.75, nan, 12.0]


### IN-CLASS ACTIVITY -- TODO #6

Use `str.contains()` with a regex to filter `df_imputed` for employees whose `PartCode` **starts with `HW`**. Print how many there are and show their Name and PartCode columns.

*Hint: The regex `^HW` matches strings that start with "HW".*

In [34]:
# TODO #6: Filter df_imputed for employees whose PartCode starts with 'HW'.
# Print the count and display their Name and PartCode.
# Your code here (1-2 lines):


---
# Summary

| Step | What we did | Key methods |
|------|------------|-------------|
| **Explore** | Grouped data, built pivot and contingency tables | `groupby()`, `pivot_table()` |
| **Structure** | Formatted types, scaled features, unpacked columns | `to_datetime()`, `StandardScaler`, `str.split()` |
| **Clean** | Found and handled missing values, outliers, duplicates | `isnull()`, `dropna()`, `fillna()`, `drop_duplicates()` |
| **Enrich** | Derived features, merged external data | `merge()`, `concat()`, `apply()`, `pd.cut()` |
| **Regex** | Extracted, filtered, and cleaned text data | `re.search()`, `str.extract()`, `str.contains()` |

### Takeaways

1. **Explore first** — understand what you have before you start changing things
2. **Standardize formats early** — saves you headaches later
3. **Discard vs. impute** depends on how much data is missing and whether the missingness is random
4. **Feature engineering** is where domain knowledge really shines
5. **Regex** is worth learning — once you know the basics, you'll reach for it all the time